# Homework: Vector Search
## Embeddings
Qdrant uses [fastembed](https://github.com/qdrant/fastembed) under the hood to turn text into vectors. We will now explore this library. Make sure it's installed:

In [1]:
# pip install fastembed

In [4]:
from fastembed import TextEmbedding
import numpy as np

# Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.<br>
You should get a numpy array of size 512.

What's the minimal value in this array?
- -0.51
- -0.11
- 0
- 0.51

In [11]:
# Query to embed
query = 'I just discovered the course. Can I join now?'

# Initialize the TextEmbedding model
embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

embeddings = list(embedding_model.embed(query))
embedding_array = embeddings[0]

In [19]:
print(f"The size of the embedding array is: {embedding_array.shape[0]}")
print(f"The minimal value in the embedding array is: {np.min(embedding_array).round(3)}")

The size of the embedding array is: 512
The minimal value in the embedding array is: -0.117


## Cosine similarity
The vectors that our embedding model returns are already normalized: their length is 1.0.
You can check that by using the norm function:

In [20]:
np.linalg.norm(embedding_array)

np.float64(1.0)

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.<br>
For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

In [21]:
embedding_array.dot(embedding_array)

np.float64(1.0000000000000002)

# Q2 Cosine similarity with another vector
Now let's embed this document:

In [22]:
doc = 'Can I still join the course after the start date?'

In [24]:
q2 = list(embedding_model.embed(doc))[0]

What's the cosine similarity between the vector for the query and the vector for the document?
- 0.3
- 0.5
- 0.7
- 0.9

In [26]:
embedding_array.dot(q2).round(1)

np.float64(0.9)

# Q3 Ranking by cosine
For Q3 and Q4 we will use these documents:

In [27]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):
- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:
```python
V.dot(q)
```
If this hint is rather confusing you than helping, feel free to ignore it.

In [31]:
embeddings = []

for doc in documents:
    embedded_array = list(embedding_model.embed(doc['text']))[0]
    embeddings.append(embedded_array)

embeddings = np.array(embeddings)

In [36]:
cosine_similarities = embeddings.dot(q2)
cosine_similarities

array([0.81060572, 0.84989067, 0.79913355, 0.68130471, 0.67392881])

In [38]:
print(f'The document with the highest similarity is at index: {np.argmax(cosine_similarities)}')

The document with the highest similarity is at index: 1


# Q4 Ranking by cosines, version two
Now let's calculate a new field, which is a concatenation of question and text:
```python
full_text = doc['question'] + ' ' + doc['text']
```

In [39]:
embeddings = []

for doc in documents:
    embedded_array = list(embedding_model.embed(doc['question'] + ' ' + doc['text']))[0]
    embeddings.append(embedded_array)

embeddings = np.array(embeddings)

In [40]:
cosine_similarities = embeddings.dot(q2)
cosine_similarities

array([0.92732523, 0.85948475, 0.83966021, 0.76320298, 0.76044493])

In [41]:
print(f'The document with the highest similarity is at index: {np.argmax(cosine_similarities)}')

The document with the highest similarity is at index: 0


It is different from Q3, because now it's taking into consideration the question, which is improves the result

# Q5 Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?
- 128
- 256
- 384
- 512
  
One of these models is BAAI/bge-small-en. Let's use it.

In [43]:
import json

dimensionalities = [128, 256, 384]

for model in TextEmbedding.list_supported_models():
    for dim in dimensionalities:
        if model['dim'] == dim:
            print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [47]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = 'BAAI/bge-small-en'

# Q6. Indexing with qdrant
For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [45]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [48]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [49]:
collection_name = "vector-search-hw"

In [50]:
qd_client.delete_collection(collection_name=collection_name)

False

In [51]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:
```python
text = doc['question'] + ' ' + doc['text']
```

In [52]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [53]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

After the data is inserted, use the question from Q1 for querying the collection.

In [54]:
query

'I just discovered the course. Can I join now?'

In [67]:
def query_points(question):
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=1,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point)
    
    return results

What's the highest score in the results? (The score for the first returned record):
- 0.97
- 0.87
- 0.77
- 0.67

In [68]:
query_points(query)

[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]

In [70]:
query_points(query)[0].score

0.8703172

In [73]:
print(f'The highest score in the results is {np.round(query_points(query)[0].score,2)}')

The highest score in the results is 0.87
